In [1]:
import json

with open("train.json") as training:
    training = json.loads(training.read())
    
with open("test.json") as testing:
    testing = json.loads(testing.read())

In [2]:
training_dict = {}
testing_dict = {}

for user in training.keys():
    for [item, rating] in training[user]:
        if user in training_dict:
            training_dict[user][item] = rating
        else:
            training_dict[user] = {item: rating}
            
for user in testing.keys():
    for [item, rating] in testing[user]:
        if user in testing_dict:
            testing_dict[user][item] = rating
        else:
            testing_dict[user] = {item: rating}


In [3]:
def sub_row_mean(ratings_dict):
    sub_row_mean_ratings_dict = {}
    for user in ratings_dict:
        list_item = list(ratings_dict[user].values())
        avg = sum(list_item) / len(list_item)
        for item in ratings_dict[user].keys():
            if user in sub_row_mean_ratings_dict:
                sub_row_mean_ratings_dict[user][item] = ratings_dict[user][item] - avg
            else:
                sub_row_mean_ratings_dict[user] = {item: ratings_dict[user][item] - avg}
    
    return sub_row_mean_ratings_dict
        


In [4]:
import numpy as np
from numpy.linalg import norm
import math

def cosine_sim(ratings_dict, user):
    cosine_sim_val = {}

    for other_user in ratings_dict.keys():
        dot_prod = sum(ratings_dict[user][item]*ratings_dict[other_user].get(item, 0) for item in ratings_dict[user])
        user_norm = np.linalg.norm(np.array([rating for rating in ratings_dict[user].values()]))
        other_user_norm = np.linalg.norm(np.array([rating for rating in ratings_dict[other_user].values()]))
        norm_mul = user_norm * other_user_norm
        if norm_mul == 0:
            cos_sim = 0
        else:
            cos_sim = dot_prod / norm_mul
        if math.isnan(cos_sim):
            cos_sim = 0
        cosine_sim_val[other_user] = cos_sim
    
        if cosine_sim_val[other_user] > 1:
            cosine_sim_val[other_user] = 1
        if cosine_sim_val[other_user] < 0:

            cosine_sim_val[other_user] = 0        
    
    return cosine_sim_val

In [5]:

def predict_item_rating_avg(ratings_dict, cos_similarities_dict, k, item, user):

    cos_similarities_dict.pop(user)
    other_users = list(cos_similarities_dict.keys())
    for other_user in other_users:
        if item not in ratings_dict[other_user]:
            cos_similarities_dict.pop(other_user)
    
    list_cos_sim = sorted(cos_similarities_dict.items(), key=lambda x: x[1], reverse=True)[:k]

    avg_rating = 0
    
    if len(list_cos_sim) != 0 and list_cos_sim[0][1] != 0:
        for pair in list_cos_sim:
            avg_rating += ratings_dict[pair[0]][item]
        
        avg_rating = avg_rating / len(list_cos_sim)
    else:
        total = 0
        rating = 0
        for rat in training_dict[user]:
            rating += training_dict[user][rat]
            total += 1
        rating = rating / total
        return rating
    
    return avg_rating


    
    

In [6]:


def predict_item_rating_sim(ratings_dict, cos_similarities_dict, k, item, user):

    if user in cos_similarities_dict:
        cos_similarities_dict.pop(user)
    other_users = list(cos_similarities_dict.keys())
    for other_user in other_users:
        if item not in ratings_dict[other_user]:
            cos_similarities_dict.pop(other_user)
    
    list_cos_sim = sorted(cos_similarities_dict.items(), key=lambda x: x[1], reverse=True)[:k]
    rating = 0
    similarities = 0
    
    if len(list_cos_sim) != 0 and list_cos_sim[0][1] != 0:
        for pair in list_cos_sim:
            rating += ratings_dict[pair[0]][item] * cos_similarities_dict[pair[0]]
            similarities += cos_similarities_dict[pair[0]]
        rating = rating / similarities
    else:
        total = 0
        rating = 0
        for rat in training_dict[user]:
            rating += training_dict[user][rat]
            total += 1
        rating = rating / total
        
    if rating > 5.1:
        print("Flag")
        print("")
        rating = 4.99
    return rating

  
    

In [7]:
def predict(ratings_dict, user, items):
    predicted_ratings_dict = {}
    sub_mean_dict = sub_row_mean(ratings_dict)
    cos_sim_dict = cosine_sim(sub_mean_dict, user)
    for item in items:
        avg_rating_prediction = predict_item_rating_avg(training_dict, cos_sim_dict.copy(), k, item, user)
        sim_rating_prediction = predict_item_rating_sim(training_dict, cos_sim_dict.copy(), k, item, user)
        
        predicted_ratings_dict[item] = (avg_rating_prediction, sim_rating_prediction)
        
    return predicted_ratings_dict

In [8]:
def first_one_in_dict(dict_first_one):
    count = 0
    for item,rating in dict_first_one.items():
        if rating == 1:
            return count
        else:
            count+=1
    return -1


In [9]:
def ndcg(recommended_items_ratings_avg, recommended_items_ratings_sim, user):
    orig_reviewed_items = (sorted(testing_dict[user].items(), key=lambda x: x[1], reverse=True)).keys()
    orig_reviewed_items_hit_avg = {}
    orig_reviewed_items_hit_sim = {}
    
    for item in orig_reviewed_items:
        orig_reviewed_items_hit_avg[item] = 0
        orig_reviewed_items_hit_sim[item] = 0
    
    recommended_items_avg = []#recommended_items_ratings_avg.keys()
    recommended_items_sim = []#recommended_items_ratings_sim.keys()

    for item in recommended_items_ratings_avg:
        recommended_items_avg.append(item[0])

    for item in recommended_items_ratings_sim:
        recommended_items_sim.append(item[0])

    same_avg_items = 0
    same_sim_items = 0

    for item in recommended_items_avg:
        if item in orig_reviewed_items:
            same_avg_items += 1
            orig_reviewed_items_hit_avg[item] = 1
            
            

    for item in recommended_items_sim:
        if item in orig_reviewed_items:
            same_sim_items += 1
            orig_reviewed_items_hit_sim[item] = 1
            
    dcg_avg = 0
    idcg_avg = 0
    ndcg_avg = 0
    if same_avg_items == 0:
        ndcg_avg = 0
    else:
        first_one = first_one_in_dict(orig_reviewed_items_hit_avg)
        for i in range(same_avg_items):
            dcg_avg += (1/math.log2(1 + first_one + i + 1))
            idcg_avg += (1/math.log2(1 + i + 1))
            
        ndcg_avg = dcg_avg/idcg_avg
        
        
    dcg_sim = 0
    idcg_sim = 0
    ndcg_sim = 0
    if same_sim_items == 0:
        ndcg_sim = 0
    else:
        first_one = first_one_in_dict(orig_reviewed_items_hit_sim)
        for i in range(same_sim_items):
            dcg_sim += (1/math.log2(1 + first_one + i + 1))
            idcg_sim += (1/math.log2(1 + i + 1))
            
        ndcg_sim = dcg_sim/idcg_sim
    print('NDCG Avg: ')
    print(ndcg_avg)
    print('NDCG Sim: ')
    print(ndcg_sim)
   
    


In [10]:
def scores(recommended_items_ratings_avg, recommended_items_ratings_sim, user):



    print((user, recommended_items_ratings_avg))
    print((user, recommended_items_ratings_sim))

    orig_reviewed_items = testing_dict[user].keys()
    recommended_items_avg = []#recommended_items_ratings_avg.keys()
    recommended_items_sim = []#recommended_items_ratings_sim.keys()
    recommended_items_pair_avg = {}#recommended_items_ratings_avg.keys()
    recommended_items_pair_sim = {}#recommended_items_ratings_sim.keys()
    

    for item in recommended_items_ratings_avg:
        recommended_items_avg.append(item[0])
        recommended_items_pair_avg[item[0]] = 0 
        

    for item in recommended_items_ratings_sim:
        recommended_items_sim.append(item[0])
        recommended_items_pair_sim[item[0]] = 0 

    same_avg_items = 0
    same_sim_items = 0

    for item in recommended_items_avg:
        if item in orig_reviewed_items:
            same_avg_items += 1
            recommended_items_pair_avg[item] = 1

    for item in recommended_items_sim:
        if item in orig_reviewed_items:
            same_sim_items += 1
            recommended_items_pair_sim[item] = 1

    precision_avg = (same_avg_items/k)
    precision_sim = (same_sim_items/k)

    print("Precision Avg: ")
    print(precision_avg)
    print('Precision Sim: ')
    print(precision_sim)


    recall_avg = (same_avg_items/len(orig_reviewed_items))
    recall_sim = (same_sim_items/len(orig_reviewed_items))

    print('Recall Avg: ')
    print(recall_avg)
    print('Recall Sim: ')
    print(recall_sim)
    if precision_avg+recall_avg == 0:
        f_score_avg = 0
    else:
        f_score_avg = (2*precision_avg*recall_avg)/(precision_avg+recall_avg)

    if precision_sim+recall_sim == 0:
        f_score_sim = 0
    else:
        f_score_sim = (2*precision_sim*recall_sim)/(precision_sim+recall_sim)

    print('F_Score Avg: ')
    print(f_score_avg)
    print('F_Score Sim: ')
    print(f_score_sim)
    print(testing_dict[user])
    

    
    dcg_avg = 0
    idcg_avg = 0
    ndcg_avg = 0
    if same_avg_items != 0:
        first_one = first_one_in_dict(recommended_items_pair_avg)
        for i in range(same_avg_items):
            dcg_avg += (1/math.log2(1 + first_one + i + 1))
            idcg_avg += (1/math.log2(1 + i + 1))
            
        ndcg_avg = dcg_avg/idcg_avg
        
        
    dcg_sim = 0
    idcg_sim = 0
    ndcg_sim = 0
    if same_sim_items != 0:
        first_one = first_one_in_dict(recommended_items_pair_sim)
        for i in range(same_sim_items):
            dcg_sim += (1/math.log2(1 + first_one + i + 1))
            idcg_sim += (1/math.log2(1 + i + 1))
            
        ndcg_sim = dcg_sim/idcg_sim
    print('NDCG Avg: ')
    print(ndcg_avg)
    print('NDCG Sim: ')
    print(ndcg_sim)
    
    return(precision_avg, precision_sim, recall_avg, recall_sim, f_score_avg, f_score_sim, ndcg_avg, ndcg_sim)


    
    
    
    
    
    
    
    

In [11]:
all_items = set()
for user,list_ratings in training_dict.items():
    all_items.update(list(training_dict[user].keys()))

for user,list_ratings in testing_dict.items():
    all_items.update(list(testing_dict[user].keys()))


In [12]:
top_n_testing_dict = dict(sorted(testing_dict.items(), key=lambda x: len(x[1]), reverse=True)[:100])

In [14]:
import time

k = 10


prec_avg, prec_sim, rec_avg, rec_sim, f_avg, f_sim, ndcgav, ndcgsim = 0, 0, 0, 0, 0, 0, 0, 0

start = time.time()
i = 0
j = 0

for user, list_ratings in top_n_testing_dict.items():
    i += 1
    predicted_vals = predict(training_dict, user, list(all_items))

    orig_vals = training_dict[user]

    for item,rating in orig_vals.items():

        predicted_vals.pop(item)
        
    print('User Count: ' + str(i))
    print(time.time() - start)


    recommended_items_ratings_avg = sorted(predicted_vals.items(), key=lambda x: x[1][0], reverse=True)[:k]
    recommended_items_ratings_sim = sorted(predicted_vals.items(), key=lambda x: x[1][1], reverse=True)[:k]

    temp_prec_avg, temp_prec_sim, temp_rec_avg, temp_rec_sim, temp_f_avg, temp_f_sim, temp_ndcgav, temp_ndcgsim = scores(recommended_items_ratings_avg, recommended_items_ratings_sim, user)
    
    prec_avg += temp_prec_avg
    prec_sim += temp_prec_sim
    rec_avg += temp_rec_avg
    rec_sim += temp_rec_sim
    f_avg += temp_f_avg
    f_sim += temp_f_sim
    ndcgav += temp_ndcgav
    ndcgsim += temp_ndcgsim




end = time.time()

print(end - start) #=> 0.0011411383748054504

print('Total Precision Average: ')
print(prec_avg/100)
print('Total Precision Similar: ')
print(prec_sim/100)
print('Total Recall Average: ')
print(rec_avg/100)
print('Total Recall Similar: ')
print(rec_sim/100)
print('Total F Score Average: ')
print(f_avg/100)
print('Total F Score Similar: ')
print(f_sim/100)
print('NDCG Average: ')
print(ndcgav/100)
print('NDCG Similar: ')
print(ndcgsim/100)

User Count: 1
543.565309047699
('A3W4D8XOGLWUN5', [('B00EDLMURS', (5.0, 5.0)), ('B003Z2VGRG', (5.0, 5.0)), ('B00N21O90Q', (5.0, 5.0)), ('B000VWQGXC', (5.0, 5.0)), ('B00EQNXCV6', (5.0, 5.0)), ('B000W0TAD6', (5.0, 5.0)), ('B0042U8TB6', (5.0, 5.0)), ('B000W16ZZQ', (5.0, 5.0)), ('B00136S34U', (5.0, 5.0)), ('B013FILLKQ', (5.0, 5.0))])
('A3W4D8XOGLWUN5', [('B0159K08KC', (5.0, 5.000000000000002)), ('B000V693C4', (4.9, 5.000000000000001)), ('B006P8EFES', (4.8, 5.000000000000001)), ('B00136JF9M', (4.9, 5.000000000000001)), ('B002KWE4MQ', (4.6, 5.000000000000001)), ('B001411WRE', (4.7, 5.000000000000001)), ('B0173O5PSC', (4.5, 5.000000000000001)), ('B001NCJACI', (4.9, 5.000000000000001)), ('B000W015NE', (4.75, 5.000000000000001)), ('B00136JSC6', (5.0, 5.000000000000001))])
Precision Avg: 
0.0
Precision Sim: 
0.0
Recall Avg: 
0.0
Recall Sim: 
0.0
F_Score Avg: 
0
F_Score Sim: 
0
{'B00A7299JY': 4.0, 'B000WLH8PW': 3.0, 'B00137GJZO': 4.0, 'B00JBTM2JS': 4.0, 'B007IR7RWC': 3.0, 'B00137KS2O': 4.0, 'B01C

User Count: 4
2063.5588302612305
('A36EDWL4F3AASU', [('B00137STOS', (5.0, 5.0)), ('B00N21O90Q', (5.0, 4.999999999999999)), ('B000VWQGXC', (5.0, 5.0)), ('B0075FLWVO', (5.0, 5.0)), ('B000W0TAD6', (5.0, 5.0)), ('B0042U8TB6', (5.0, 5.0)), ('B000W16ZZQ', (5.0, 5.0)), ('B013FILLKQ', (5.0, 5.0)), ('B007IE7RT8', (5.0, 5.0)), ('B00J6155L8', (5.0, 5.0))])
('A36EDWL4F3AASU', [('B000VZSBPK', (5.0, 5.000000000000002)), ('B002LCAZQE', (4.8, 5.000000000000001)), ('B001NCJACI', (4.9, 5.000000000000001)), ('B003A9CK92', (4.666666666666667, 5.000000000000001)), ('B003UPGHTK', (4.8, 5.000000000000001)), ('B00U370GS6', (4.6, 5.000000000000001)), ('B00137XHV8', (4.8, 5.000000000000001)), ('B00138DFP0', (5.0, 5.000000000000001)), ('B000W178XO', (4.9, 5.000000000000001)), ('B0013TQNC6', (5.0, 5.000000000000001))])
Precision Avg: 
0.0
Precision Sim: 
0.0
Recall Avg: 
0.0
Recall Sim: 
0.0
F_Score Avg: 
0
F_Score Sim: 
0
{'B001VW9UES': 1.0, 'B0042U8VUA': 5.0, 'B00138FSPU': 1.0, 'B001NTPENA': 4.0, 'B00W4SVHYE': 

User Count: 9
4103.933300495148
('A15T3Y3BL2AJ9V', [('B00EDLMURS', (5.0, 5.0)), ('B00137STOS', (5.0, 5.0)), ('B00BU2UANO', (5.0, 5.0)), ('B001380PWQ', (5.0, 5.0)), ('B0042U8TB6', (5.0, 5.0)), ('B002TG4DZQ', (5.0, 5.0)), ('B0011XHE4K', (5.0, 5.0)), ('B00123BAMQ', (5.0, 5.0)), ('B00FXES7XK', (5.0, 5.0)), ('B000X0ER60', (5.0, 5.0))])
('A15T3Y3BL2AJ9V', [('B00G2IAIVK', (5.0, 5.000000000000002)), ('B00137IKD8', (5.0, 5.000000000000001)), ('B000SX85CE', (4.9, 5.000000000000001)), ('B000ZN0ZZC', (5.0, 5.000000000000001)), ('B000SZHZIW', (5.0, 5.000000000000001)), ('B00OXE3MO0', (4.6, 5.000000000000001)), ('B00Y1RFVD4', (5.0, 5.000000000000001)), ('B00124AA2G', (4.8, 5.000000000000001)), ('B000TE4TYA', (4.875, 5.000000000000001)), ('B001223FFC', (4.666666666666667, 5.000000000000001))])
Precision Avg: 
0.0
Precision Sim: 
0.0
Recall Avg: 
0.0
Recall Sim: 
0.0
F_Score Avg: 
0
F_Score Sim: 
0
{'B000WOUSQK': 5.0, 'B001NY8HUW': 5.0, 'B0013JWR6W': 5.0, 'B001388QDG': 5.0, 'B000TDYM7K': 4.0, 'B001CJO

User Count: 15
6513.755841493607
('A2PR1O53J0V8W2', [('B00EDLMURS', (5.0, 5.0)), ('B000TD779K', (5.0, 5.0)), ('B001458LW4', (5.0, 5.0)), ('B001KQJWAM', (5.0, 5.0)), ('B00QYGUOI2', (5.0, 5.0)), ('B00137STOS', (5.0, 5.0)), ('B00WZULC5U', (5.0, 5.0)), ('B001KQGEG2', (5.0, 5.0)), ('B00B931GMI', (5.0, 5.0)), ('B000V693C4', (5.0, 5.0))])
('A2PR1O53J0V8W2', [('B00EDLMURS', (5.0, 5.0)), ('B000TD779K', (5.0, 5.0)), ('B001458LW4', (5.0, 5.0)), ('B001KQJWAM', (5.0, 5.0)), ('B00QYGUOI2', (5.0, 5.0)), ('B00137STOS', (5.0, 5.0)), ('B00WZULC5U', (5.0, 5.0)), ('B001KQGEG2', (5.0, 5.0)), ('B00B931GMI', (5.0, 5.0)), ('B000V693C4', (5.0, 5.0))])
Precision Avg: 
0.0
Precision Sim: 
0.0
Recall Avg: 
0.0
Recall Sim: 
0.0
F_Score Avg: 
0
F_Score Sim: 
0
{'B00138DNWA': 5.0, 'B00ZRRB8WA': 5.0, 'B009Y6V44W': 5.0, 'B001OGNMPY': 5.0, 'B00D7OTNIG': 5.0, 'B00136LPY0': 5.0, 'B00136LVM6': 5.0, 'B000VZIN6W': 5.0, 'B00V8Z8SJ6': 5.0, 'B001NYRPM8': 5.0, 'B0011ZR5CO': 5.0, 'B000QNZNNA': 5.0, 'B001R6W978': 5.0, 'B00LPQPA1C

User Count: 21
8922.518340587616
('A3GJ6CFWN47N0U', [('B00EDLMURS', (5.0, 5.0)), ('B000TD779K', (5.0, 5.0)), ('B001458LW4', (5.0, 5.0)), ('B001KQJWAM', (5.0, 5.0)), ('B00QYGUOI2', (5.0, 5.0)), ('B00137STOS', (5.0, 5.0)), ('B00WZULC5U', (5.0, 5.0)), ('B001KQGEG2', (5.0, 5.0)), ('B00B931GMI', (5.0, 5.0)), ('B000V693C4', (5.0, 5.0))])
('A3GJ6CFWN47N0U', [('B00EDLMURS', (5.0, 5.0)), ('B000TD779K', (5.0, 5.0)), ('B001458LW4', (5.0, 5.0)), ('B001KQJWAM', (5.0, 5.0)), ('B00QYGUOI2', (5.0, 5.0)), ('B00137STOS', (5.0, 5.0)), ('B00WZULC5U', (5.0, 5.0)), ('B001KQGEG2', (5.0, 5.0)), ('B00B931GMI', (5.0, 5.0)), ('B000V693C4', (5.0, 5.0))])
Precision Avg: 
0.0
Precision Sim: 
0.0
Recall Avg: 
0.0
Recall Sim: 
0.0
F_Score Avg: 
0
F_Score Sim: 
0
{'B002Q5FVME': 5.0, 'B0012F6LL4': 5.0, 'B000V630L4': 5.0, 'B017JXT50G': 5.0, 'B00124B4B2': 5.0, 'B00123K086': 5.0, 'B000VWKJX0': 5.0, 'B001O7UX3W': 5.0, 'B000V64KJU': 5.0, 'B00136J8K8': 5.0, 'B00137G8MS': 5.0, 'B000V64KJK': 5.0, 'B00136LHZ2': 5.0, 'B00136PQSQ

...

User Count: 67
28026.724744558334
('AAFW69F6T4CIX', [('B001380PWQ', (5.0, 5.0)), ('B013FILLKQ', (5.0, 5.0)), ('B0153T25NC', (5.0, 5.0)), ('B01F9S5ICC', (5.0, 5.0)), ('B00J6155L8', (5.0, 5.000000000000001)), ('B00FXES7XK', (5.0, 5.0)), ('B000WQNGHG', (5.0, 5.0)), ('B00XXL7894', (5.0, 5.0)), ('B004B2RVDM', (5.0, 5.0)), ('B002QWH8NW', (5.0, 4.999999999999999))])
('AAFW69F6T4CIX', [('B00J6155L8', (5.0, 5.000000000000001)), ('B00U370GS6', (4.6, 5.000000000000001)), ('B00UCQ9EY0', (5.0, 5.000000000000001)), ('B00122RQX4', (4.75, 5.000000000000001)), ('B00JXJA8O2', (4.9, 5.000000000000001)), ('B0022V2062', (5.0, 5.000000000000001)), ('B016RWV6BG', (4.777777777777778, 5.000000000000001)), ('B000VZSBPK', (5.0, 5.000000000000001)), ('B017K90F4O', (5.0, 5.000000000000001)), ('B00A8AJML0', (5.0, 5.000000000000001))])
Precision Avg: 
0.0
Precision Sim: 
0.1
Recall Avg: 
0.0
Recall Sim: 
0.05
F_Score Avg: 
0
F_Score Sim: 
0.06666666666666667
{'B00U370GS6': 5.0, 'B000W23918': 5.0, 'B0170K9WTM': 5.0, 

User Count: 74
31016.857920646667
('AIZWM3J1HG027', [('B00EDLMURS', (5.0, 5.0)), ('B000TD779K', (5.0, 5.0)), ('B001458LW4', (5.0, 5.0)), ('B001KQJWAM', (5.0, 5.0)), ('B00QYGUOI2', (5.0, 5.0)), ('B00137STOS', (5.0, 5.0)), ('B00WZULC5U', (5.0, 5.0)), ('B001KQGEG2', (5.0, 5.0)), ('B00B931GMI', (5.0, 5.0)), ('B000V693C4', (5.0, 5.0))])
('AIZWM3J1HG027', [('B00EDLMURS', (5.0, 5.0)), ('B000TD779K', (5.0, 5.0)), ('B001458LW4', (5.0, 5.0)), ('B001KQJWAM', (5.0, 5.0)), ('B00QYGUOI2', (5.0, 5.0)), ('B00137STOS', (5.0, 5.0)), ('B00WZULC5U', (5.0, 5.0)), ('B001KQGEG2', (5.0, 5.0)), ('B00B931GMI', (5.0, 5.0)), ('B000V693C4', (5.0, 5.0))])
Precision Avg: 
0.0
Precision Sim: 
0.0
Recall Avg: 
0.0
Recall Sim: 
0.0
F_Score Avg: 
0
F_Score Sim: 
0
{'B004L9BAL4': 5.0, 'B0092YOKG0': 5.0, 'B004CE6NIS': 5.0, 'B00K4HKUHM': 5.0, 'B004L9D86Y': 5.0, 'B008UX83BC': 5.0, 'B004L99OMQ': 5.0, 'B00932IOQ8': 5.0, 'B004CE6NE2': 5.0, 'B007G0RUFU': 5.0, 'B00EYAMFVE': 5.0, 'B00136JGZU': 5.0, 'B00W4IGBGS': 5.0, 'B00CG68C0G'

User Count: 81
33889.6255505085
('ADHZ0ABZBBDFJ', [('B00EDLMURS', (5.0, 5.0)), ('B000TD779K', (5.0, 5.0)), ('B001458LW4', (5.0, 5.0)), ('B001KQJWAM', (5.0, 5.0)), ('B00QYGUOI2', (5.0, 5.0)), ('B00137STOS', (5.0, 5.0)), ('B00WZULC5U', (5.0, 5.0)), ('B001KQGEG2', (5.0, 5.0)), ('B00B931GMI', (5.0, 5.0)), ('B000V693C4', (5.0, 5.0))])
('ADHZ0ABZBBDFJ', [('B00EDLMURS', (5.0, 5.0)), ('B000TD779K', (5.0, 5.0)), ('B001458LW4', (5.0, 5.0)), ('B001KQJWAM', (5.0, 5.0)), ('B00QYGUOI2', (5.0, 5.0)), ('B00137STOS', (5.0, 5.0)), ('B00WZULC5U', (5.0, 5.0)), ('B001KQGEG2', (5.0, 5.0)), ('B00B931GMI', (5.0, 5.0)), ('B000V693C4', (5.0, 5.0))])
Precision Avg: 
0.0
Precision Sim: 
0.0
Recall Avg: 
0.0
Recall Sim: 
0.0
F_Score Avg: 
0
F_Score Sim: 
0
{'B00136LDKQ': 5.0, 'B00136RS8C': 5.0, 'B00136J8K8': 5.0, 'B00137MFXY': 5.0, 'B00136LITC': 5.0, 'B00136JI5S': 5.0, 'B00136RO2M': 5.0, 'B0033UL45O': 5.0, 'B000W239UY': 5.0, 'B000VSWCQG': 5.0, 'B00136PRPS': 5.0, 'B00136LG4O': 5.0, 'B000SZE0CG': 5.0, 'B000W1T2TM': 

User Count: 88
36269.1497092247
('A2R35DE00SQDJQ', [('B00G5X9XEK', (5.0, 5.0)), ('B00GNCJA2I', (5.0, 5.0)), ('B00ETQ9Q6K', (5.0, 5.0)), ('B00ONGFA66', (5.0, 5.0)), ('B00ROK0FVS', (5.0, 5.0)), ('B00A19O78G', (5.0, 5.0)), ('B00QM9MZA6', (5.0, 5.0)), ('B00LPWXAA4', (5.0, 5.0)), ('B00K4H9JMO', (5.0, 5.0)), ('B00DFQDDQ4', (5.0, 5.0))])
('A2R35DE00SQDJQ', [('B00G5X9XEK', (5.0, 5.0)), ('B00GNCJA2I', (5.0, 5.0)), ('B00ETQ9Q6K', (5.0, 5.0)), ('B00ONGFA66', (5.0, 5.0)), ('B00ROK0FVS', (5.0, 5.0)), ('B00A19O78G', (5.0, 5.0)), ('B00L4L8MCW', (4.5, 5.0)), ('B00QM9MZA6', (5.0, 5.0)), ('B00LPWXAA4', (5.0, 5.0)), ('B000WZWW7M', (4.9, 5.0))])
Precision Avg: 
0.0
Precision Sim: 
0.0
Recall Avg: 
0.0
Recall Sim: 
0.0
F_Score Avg: 
0
F_Score Sim: 
0
{'B00JI68EGE': 5.0, 'B01E8489EM': 5.0, 'B0164CS1IK': 5.0, 'B005D1G6TK': 5.0, 'B005BIBRVC': 5.0, 'B00GHG5524': 5.0, 'B00N5WHFHG': 5.0, 'B01ECSVMBQ': 5.0, 'B01BSA3V42': 5.0, 'B00DJRGADC': 5.0, 'B011N52BYI': 5.0, 'B014JXV41M': 5.0, 'B00IZIEFTQ': 5.0, 'B01FN755HW'

User Count: 95
38382.72993969917
('A3I7J6H0FSZQIW', [('B00EDLMURS', (5.0, 5.0)), ('B000TD779K', (5.0, 5.0)), ('B001458LW4', (5.0, 5.0)), ('B001KQJWAM', (5.0, 5.0)), ('B00QYGUOI2', (5.0, 5.0)), ('B00137STOS', (5.0, 5.0)), ('B00WZULC5U', (5.0, 5.0)), ('B001KQGEG2', (5.0, 5.0)), ('B00B931GMI', (5.0, 5.0)), ('B000V693C4', (5.0, 5.0))])
('A3I7J6H0FSZQIW', [('B00EDLMURS', (5.0, 5.0)), ('B000TD779K', (5.0, 5.0)), ('B001458LW4', (5.0, 5.0)), ('B001KQJWAM', (5.0, 5.0)), ('B00QYGUOI2', (5.0, 5.0)), ('B00137STOS', (5.0, 5.0)), ('B00WZULC5U', (5.0, 5.0)), ('B001KQGEG2', (5.0, 5.0)), ('B00B931GMI', (5.0, 5.0)), ('B000V693C4', (5.0, 5.0))])
Precision Avg: 
0.0
Precision Sim: 
0.0
Recall Avg: 
0.0
Recall Sim: 
0.0
F_Score Avg: 
0
F_Score Sim: 
0
{'B005XT4CAW': 5.0, 'B005C0WM7M': 5.0, 'B00123HEWQ': 5.0, 'B00136RUWG': 5.0, 'B0013EVPN8': 5.0, 'B001NZXIO6': 5.0, 'B00330WF7K': 5.0, 'B001NYVYAW': 5.0, 'B00973F588': 5.0, 'B003P07RVM': 5.0, 'B001A7BNZI': 5.0, 'B00122CB70': 5.0, 'B00137YP4G': 5.0, 'B00181OEZ2